In [1]:
model = "llama3.2"

#### Task 1: Create a Simple Chain for Summarization (Erstellen Sie eine einfache Kette zur Zusammenfassung)

**Objective:**

Build a LangChain chain that can summarize a given text. (Erstellen Sie eine LangChain-Kette, die einen bestimmten Text zusammenfassen kann.)

**Task Description:**

- Create a llm chain using with a Ollama model. (Erstellen Sie eine LLM-Kette mithilfe eines Ollama-Modells.)
- Define a prompt template for summarization. The summary should be only one sentence. (Definieren Sie eine Eingabeaufforderungsvorlage für die Zusammenfassung. Die Zusammenfassung sollte nur einen Satz umfassen.)
- Run the chain with a sample text and print the summary. (Führen Sie die Kette mit einem Beispieltext aus und drucken Sie die Zusammenfassung aus.)
- Add model output streaming. (Modellausgabe-Streaming hinzufügen.)
- Run the chain with streaming with a sample text and print the summary. (Führen Sie die Kette mit Streaming mit einem Beispieltext aus und drucken Sie die Zusammenfassung aus.)

**Useful links:**

- [How To Prompt Template 1](https://python.langchain.com/v0.2/docs/tutorials/extraction/#the-extractor)
- [How To Prompt Template 2](https://python.langchain.com/v0.2/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate)
- [How To LCEL Chains 1](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel)
- [How To LCEL Chains 2](https://python.langchain.com/v0.2/docs/versions/migrating_chains/llm_chain/#lcel)
- [How To Chain Streaming](https://python.langchain.com/v0.2/docs/concepts/#streaming)


In [2]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# Load the Ollama model
llm = ChatOllama(model = model)

# ADD HERE YOUR CODE
# Define the prompt template
summarization_prompt = ChatPromptTemplate([
    ("system", "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",),
    ("human", "{text}"),
])

# ADD HERE YOUR CODE
# Create the LLMChain
summarization_chain = LLMChain(llm = llm, prompt = summarization_prompt)

# Sample text
text = """Over the last decade, deep learning has evolved massively to process and generate unstructured data like text, images, and video. 
These advanced AI models have gained popularity in various industries, and include large language models (LLMs). 
There is currently a significant level of fanfare in both the media and the industry surrounding AI,
and there’s a fair case to be made that Artificial Intelligence (AI), with these advancements,
is about to have a wide-ranging and major impact on businesses, societies, and individuals alike.
This is driven by numerous factors, including advancements in technology, high-profile applications, 
and the potential for transfor- mative impacts across multiple sectors."""

# ADD HERE YOUR CODE
# Invoke the chain
summary = summarization_chain.run(text)
print(summary)
# print(summary.content)

C:\Users\sschn\AppData\Local\Temp\ipykernel_1056\1532245166.py:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  summarization_chain = LLMChain(llm = llm, prompt = summarization_prompt)
C:\Users\sschn\AppData\Local\Temp\ipykernel_1056\1532245166.py:33: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  summary = summarization_chain.run(text)


Here are the extracted attributes:

* decade: 1 (last decade)
* data: "unstructured" ("text", "images", and "video")
* model: "large language models"
* impact: "wide-ranging and major" 
* factor: 3 (advancements in technology, high-profile applications, and transformative impacts)


In [3]:
# Stream the chain output
for chunk in summarization_chain.stream({"text": text}):
    print(chunk, end = "", flush = True)

{'text': 'Here are the extracted information:\n\n* Industry: Multiple industries\n* Type of AI models: Large language models (LLMs)\n* Impact of AI on businesses, societies, and individuals: Major impact'}

#### Task 2: Chain with Tool Usage (Simple Math Tool)

**Objective:**

Create a LangChain chain that uses a simple math tool to perform calculations.

**Task Description:**

- Define a function as tool which multiplies two integer values and return the result.
- Create a chain
- Print the result of the calculation.

**Useful links:**

- [How To Tools](https://python.langchain.com/v0.2/docs/how_to/tools_chain/#create-a-tool)
- [How To Tools in Chains](https://python.langchain.com/v0.2/docs/how_to/tools_chain/#chains)
- [How To Tool Calling](https://python.langchain.com/v0.2/docs/concepts/#functiontool-calling)
- [How To Chain and Call Tools with Ollama](https://python.langchain.com/v0.2/docs/integrations/chat/ollama/)


In [4]:
from langchain_core.tools import tool

# ADD HERE YOUR CODE
# Create custom tool
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multipliziert zwei ganze Zahlen."""
    return first_int * second_int

print(multiply.name)
print(multiply.description)
print(multiply.args) # -> definition of tool arguments

# Invoke custom tool
multiply.invoke({"first_int": 4, "second_int": 5})

multiply
Multipliziert zwei ganze Zahlen.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


20

In [5]:
# Load the Ollama model
llm = ChatOllama(model = model)

# ADD HERE YOUR CODE
# Use bind_tools to pass the definition of our tool in as part of each call to the model, so that the model can invoke the tool
llm_with_tools = llm.bind_tools([multiply])

# When the model invokes the tool, this will show up in the AIMessage.tool_calls attribute of the output -> extract tool parameters from input text
msg = llm_with_tools.invoke("whats 5 times forty two")
msg.tool_calls

[{'name': 'multiply',
  'args': {'first_int': '42', 'second_int': '5'},
  'id': 'f956523f-8e6f-41b9-972e-a2bd198a52d5',
  'type': 'tool_call'}]

In [6]:
# ADD HERE YOUR CODE
# Create the chain: pass the extracte tool parameters from the input text to the tool -> extract the arguments of the first tool_call
chain_with_tools = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | multiply

# Run chain
chain_with_tools.invoke("whats 5 times forty two")

210

#### Task 3: Agent with Tool Usage (Two Tools)

**Objective:**

Create a LangChain agent that uses two tools to perform tasks. (Erstellen Sie einen LangChain-Agenten, der zwei Tools zum Ausführen von Aufgaben verwendet.)

**Task Description:**

- Define prompt template. (Eingabeaufforderungsvorlage definieren.)
- Define tools. (Werkzeuge definieren.)
- Create an Agent using the Ollama model, prompt template and tools. (Erstellen Sie einen Agenten mithilfe des Ollama-Modells, der Eingabeaufforderungsvorlage und der Tools.)
- Run the agent with a prompt that requires one or both tools. (Führen Sie den Agenten mit einer Eingabeaufforderung aus, die eines oder beide Tools erfordert.)
- Observe how the agent uses the tools to complete the task. (Beobachten Sie, wie der Agent die Tools verwendet, um die Aufgabe abzuschließen.)

**Useful links:**

- [How To 1](https://python.langchain.com/v0.2/docs/concepts/#agents)
- [How To 2](https://python.langchain.com/v0.2/docs/how_to/tools_chain/#agents)
- [How To 3](https://python.langchain.com/v0.2/docs/tutorials/agents/)


In [7]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

# Load the Ollama model
llm = ChatOllama(model = model)

# ADD HERE YOUR CODE
agent_prompt = ChatPromptTemplate.from_template(
    """
    {agent_scratchpad}  
    You are an AI assistant. Use the provided tools to complete the user's request.
    Here is the user's request:
    {user_input}
    """)

# Custom math tools
@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


tools = [add, exponentiate]

In [8]:
# ADD HERE YOUR CODE
# Construct the tool calling agent
agent_with_tools = create_tool_calling_agent(llm, tools, agent_prompt)

# ADD HERE YOUR CODE
# Create an agent executor by passing in the agent and tools
agent_executor_with_tools = AgentExecutor(agent = agent_with_tools, tools = tools, verbose = True)

In [9]:
agent_executor_with_tools.invoke(
    {
        "user_input": "Take 3 to the fifth power then add that 12."
    }
)



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `add` with `{'first_int': '9', 'second_int': '12'}`


21
Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `add` with `{'first_int': '243', 'second_int': '12'}`


255
Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `add` with `{'first_int': '243', 'second_int': '12'}`


255
Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `add` with `{'first_int': '255', 'second_int': '12'}`


267

KeyboardInterrupt: 

#### [Optional] Task 4: Enhance Agent with Memory

**Objective:**

Eenhance the agent from Task 3 with memory to improve its context awareness and ability to maintain state.
(Erweitern Sie den Agenten aus Aufgabe 3 mit dem Gedächtnis, um sein Kontextbewusstsein und seine Fähigkeit, den Zustand aufrechtzuerhalten, zu verbessern.)

**Instructions:**

- Create a ConversationBufferMemory to store chat history. (Erstellen Sie ein ConversationBufferMemory zum Speichern des Chatverlaufs.)
- Modify the agent to use the memory to inform its responses. (Ändern Sie den Agenten so, dass er den Speicher zur Information über seine Antworten verwendet.)
- Run the agent with a series of prompts that require context or state to be maintained. (Führen Sie den Agenten mit einer Reihe von Eingabeaufforderungen aus, die die Beibehaltung des Kontexts oder Status erfordern.)
- Observe how the agent's responses improve with the addition of memory. (Beobachten Sie, wie sich die Antworten des Agenten durch die Hinzufügung von Speicher verbessern.)

**Useful links:**

- [How To Memory 1](https://python.langchain.com/v0.2/api_reference/langchain/memory/langchain.memory.buffer.ConversationBufferMemory.html#langchain.memory.buffer.ConversationBufferMemory)
- [How To Memory 1](https://python.langchain.com/v0.2/docs/versions/migrating_chains/conversation_chain/#legacy)


In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import MessagesPlaceholder

# Load the Ollama model
llm = ChatOllama(model = model)

# Define memory object for conversation history
memory = ConversationBufferMemory(memory_key = "chat_history", return_messages = True, output_key = "output")

# ADD HERE YOUR CODE
# Add history placeholder to prompt
agent_prompt_with_memory = agent_prompt

# ADD HERE YOUR CODE
# Construct the tool calling agent
agent_with_tools_and_memory = agent_with_tools

# ADD HERE YOUR CODE
# Create an agent executor by passing in the agent and tools
agent_executor_with_tools_and_memory = AgentExecutor(agent = agent_with_tools_and_memory, tools = tools, verbose = True)

In [ ]:
question = "Take 3 to the fifth power then add that 12?"                                #„Nehmen Sie 3 hoch fünf und addieren Sie dann 12?“
ai_msg_1 = agent_executor_with_tools_and_memory.invoke({"user_input": question})
print(ai_msg_1["output"])

second_question = "Explain how you have calculated the result."                         #„Erklären Sie, wie Sie das Ergebnis berechnet haben.“
ai_msg_2 = agent_executor_with_tools_and_memory.invoke({"user_input": second_question})
print(ai_msg_2["output"])



> Entering new AgentExecutor chain...

Invoking: `add` with `{'first_int': 3, 'properties': {'first_int': {'description': '', 'type': 'integer'}, 'second_int': {'description': '', 'type': 'integer'}}, 'second_int': 5}`


8
Invoking: `exponentiate` with `{'base': '3', 'exponent': '5', 'second_int': '12'}`


243
Invoking: `exponentiate` with `{'base': '3', 'exponent': '5', 'second_int': '12'}`


243
Invoking: `power` with `{'first_int': '3', 'second_int': '5'}`


power is not a valid tool, try one of [add, exponentiate].
Invoking: `power` with `{'first_int': '3', 'second_int': '5'}`


power is not a valid tool, try one of [add, exponentiate].The tool call chunk for "exponentiate" would be:
```
{
  "name": "exponentiate",
  "args": '{"base": "3", "exponent": "5", "second_int": "12"}'
}
```
So, the response to the user's request would be:
```
power is not a valid tool, try one of [add, exponentiate].
```

> Finished chain.
The tool call chunk for "exponentiate" would be:
```
{
  "name": 